In [ ]:
proj_path='/kedro-poc/test_repo/'

In [ ]:
import subprocess
import yaml

with open(proj_path+"/conf/base/catalog.yml", 'r') as stream:
    catalog = yaml.safe_load(stream)
catalog

## Params

In [ ]:
with open(proj_path+"/conf/base/parameter.yml", 'r') as stream:
    parameters = yaml.safe_load(stream)

In [ ]:
parameters

In [ ]:
import pandas
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from pyspark import SparkContext
from pyspark.sql import SparkSession, HiveContext

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

def data_acquisition(params, data):
    #data = pd.read_csv(r"data/01_raw/diabetes.csv")
    raw_data = data
    print(data.loc[[1]])
    return raw_data 

def feature_engineering(params, raw_data):
    df = raw_data
    df1 = df.dropna()
    cleaned_data = df1
    return cleaned_data

def train_test_split_fun(params, cleaned_data):
    value = params['ratio']
    training_data = cleaned_data.drop('Outcome',axis=1)
    testing_data = cleaned_data['Outcome']
    X_train, X_test, y_train, y_test = train_test_split(training_data, testing_data, test_size = value, random_state=42)
    return X_train, X_test, y_train, y_test

def model_training(params, X_train, y_train):
    rc = RandomForestClassifier(max_depth=2, random_state=0)
    model = rc.fit(X_train, y_train)
    return model


from sklearn.metrics import accuracy_score

def validation(params, rc, X_test, y_test):
    y_pred = rc.predict(X_test)
    metrics = accuracy_score(y_test,y_pred)
    print("Metrics = ",metrics)
    return metrics

In [ ]:
from kedro.pipeline import node, Pipeline
from kedro.runner import SequentialRunner
training_pipeline = Pipeline([node(data_acquisition,
                                  inputs=['parameters','diabetes'],
                                  outputs='dataset',
                                  name="data_acquisition"),
                     node(feature_engineering,
                                 inputs=['parameters','dataset'],
                                 outputs='cleaned_dataset',
                                 name="feature_engineering"),
                    node(train_test_split_fun,
                                 inputs=['parameters','cleaned_dataset'],
                                 outputs=['x_train','x_test','y_train','y_test'],
                                 name="train_test_split_fun"),
                     node(model_training,
                                 inputs=['parameters','x_train', 'y_train'],
                                 outputs='model',
                                 name="model_training"),
                                    ])

In [ ]:
from kedro.runner import SequentialRunner
from kedro.io import DataCatalog, MemoryDataSet

data_catalog = DataCatalog.from_config(catalog)
data_catalog.add_feed_dict({'parameters' : parameters})                  
SequentialRunner().run(pipeline=training_pipeline, catalog=data_catalog)  